In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from scipy import stats
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install xlsxwriter
pd.options.mode.chained_assignment = None  # default='warn'
import statsmodels.api as sm
from statsmodels.formula.api import ols
## https://www.kaggle.com/code/alexmaszanski/two-way-anova-with-python/notebook



**There are four assumptions that must be met before using two-way ANOVA:**

- Normality: Observations from the sample population are normally distributed.
- Sample Size: The number of observations must be the same for each group.
- Equal Variances: The variances for each group are equal.
- Independence: Observations in each group are independent.

In [2]:
currdir= os.getcwd()
parent = os.path.dirname(currdir)
gparent=os.path.dirname(parent)
lines_to_skip = 10 # adjust this as necessary

# count the number of header lines
header_lines = 3 # adjust this as necessary

plate_type = 'DIV19'
# read the csv file into a pandas DataFrame, skipping the metadata at the top
# df = pd.read_csv(filename, skiprows=lines_to_skip, header=[i for i in range(header_lines)])
### read all data

firstTableHeading = "Mean Firing Rate (Hz)"
basal_df= pd.read_csv(f"data/Div19_2way/Div19 H11 5th batch of gRNAs plate2 basal(000)(000)_CompiledData(1).csv",
                    skiprows = lines_to_skip)

bicucilin_df = pd.read_csv(f"data/Div19_2way/Div19 H11 5th batch of gRNAs plate2 bicu 6 microM(000)(000)_CompiledData.csv",
                         skiprows = lines_to_skip)

basal_df = basal_df.drop(columns=['Unnamed: 9']) # , 'Unnamed: 0'
bicucilin_df = bicucilin_df.drop(columns=['Unnamed: 9']) # , 'Unnamed: 0'
basal_df

,Unnamed: 0,GPR37L 1,GPR37L 2,LGI2 A12,SLITRK5,THSD7 1,THSD7 2,unt1,unt2
0,B Mean,2.588585,3.565526,2.111001,4.502087,2.431135,2.403902,6.079108,2.946073
1,B SEM,0.581044,0.602685,0.449186,0.692605,0.542782,0.566563,0.562120,0.608479
2,B Replicates,1.827421,3.278172,5.325334,3.735184,0.840568,3.100167,3.665484,2.135017
3,NaN,0.901503,6.809265,0.281511,1.074499,4.000626,3.151920,3.726210,1.878548
4,NaN,7.349958,2.711603,1.391903,2.271077,2.575334,2.032137,7.272120,2.567195
...,...,...,...,...,...,...,...,...,...
265,NaN,8.000000,7.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
266,NaN,8.000000,8.000000,7.000000,8.000000,8.000000,8.000000,8.000000,8.000000
267,NaN,8.000000,8.000000,7.000000,8.000000,8.000000,8.000000,8.000000,8.000000
268,NaN,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000


In [3]:
def rename_row_names(experiment_df):
    replace_rows = experiment_df['Unnamed: 0'].isin(['B Replicates', np.nan]) 
    replace_count = replace_rows.sum()
    
    replicates = (f"B Replicate {i+1}" for i in range(replace_count))
    with pd.option_context('mode.chained_assignment', None):
        experiment_df.loc[replace_rows, 'Unnamed: 0'] = list(replicates)
    return experiment_df

In [29]:
def section_data(exp_num, firstHeading):
    maxrows = exp_num.shape[0]
    vals_between_tables = 16
    titles = [heading_ind for heading_ind in range(14, maxrows+16, 16)]
    # print(titles[-1], print(exp_num.iloc[254]))
    # print(maxrows, titles)
    # print(maxrows, titles)
    titles.insert(0,0)

    # # Empty dictionary to store dataframes
    experiment = {}

    # Loop through start indices
    for ind in range(len(titles) -1):
        if ind == 0:
            ## Handle mean firing rate
            key = firstHeading
            table = exp_num.iloc[titles[ind] : titles[ind +1], :]
            table = rename_row_names(table)
        elif ind != 0:
            table = exp_num.iloc[titles[ind] : titles[ind +1], :]

            key = table.iloc[0][0]
            table = exp_num.iloc[titles[ind] +2: titles[ind +1], :]

            table = rename_row_names(table)
        table.set_index('Unnamed: 0', inplace=True)
        experiment[key] = table
    return experiment                   
basal_dict = section_data(basal_df, firstTableHeading)
bicucilin_dict = section_data(bicucilin_df, firstTableHeading)
bicucilin_dict


{'Mean Firing Rate (Hz)':                  GPR37L 1   GPR37L 2  LGI2 A12    SLITRK5    THSD7 1  \
 Unnamed: 0                                                             
 B Mean           7.508198   7.566295  5.616281   8.029491   7.590281   
 B SEM            0.794623   0.899518  0.690536   1.038530   0.980080   
 B Replicate 1    5.077741   2.491455  8.207586   5.371613   1.761567   
 B Replicate 2    6.225302  12.703835  2.424969   6.746978   6.615048   
 B Replicate 3    8.508754  10.762818  6.414339   3.989579  15.223218   
 B Replicate 4    6.263860  10.959775  6.397666  12.926011   9.781367   
 B Replicate 5   14.386411   9.874323  3.870988   6.783660   9.676949   
 B Replicate 6    4.979158   4.579408  0.177574   8.616715   8.996040   
 B Replicate 7    7.986244   4.357649  5.392664   7.732597   4.645269   
 B Replicate 8    7.353897   8.529804  5.844935   6.531680   7.002084   
 B Replicate 9    5.892247   6.117132  8.425594   6.661734   4.601084   
 B Replicate 10   5.920383

In [5]:
def remove_mean_SEM(dictionary, chemical_type):
    for key, value in dictionary.items():
        to_drop = [index for index in ['B Mean', 'B SEM'] if index in value.index]
        if to_drop:
            dictionary[key] = value.drop(index=to_drop)
        dictionary[key]['Chemical_Type']=chemical_type
    return dictionary

basal_dict = remove_mean_SEM(basal_dict, chemical_type="basal")
bicucilin_dict = remove_mean_SEM(bicucilin_dict, chemical_type="bicucilin")

all_dicts = dict(zip(["basal", "bicucilin"],[basal_dict, bicucilin_dict]))
all_dicts.keys()

dict_keys(['basal', 'bicucilin'])

In [6]:
def combine_chem_types(dict_of_alldicts):
    all_concat = {key: None for key in dict_of_alldicts['basal']}
    for type in dict_of_alldicts.keys():
        for test_parameter, df in dict_of_alldicts[type].items():
            all_concat[test_parameter]=pd.concat([all_concat[test_parameter], df], ignore_index=True)
    for testparam, df in all_concat.items():
        df.columns = df.columns.str.replace(' ', '_')
        for gene in df.columns:
            if gene != "Chemical_Type":
                df[gene]= df[gene].astype(float)
    return all_concat

TestParams_dict = combine_chem_types(all_dicts)


In [30]:
data = TestParams_dict['Area Under Normalized Cross-Correlation']

In [32]:
data

,GPR37L_1,GPR37L_2,LGI2_A12,SLITRK5,THSD7_1,THSD7_2,unt1,unt2,Chemical_Type
0,0.809806,0.721212,0.624224,0.652961,0.768891,0.591735,0.765553,0.780628,basal
1,0.890126,0.862841,0.727744,0.637842,0.657942,0.489055,0.739974,0.908272,basal
2,0.682422,0.872551,0.805635,0.810253,0.724886,0.887745,0.815019,0.851354,basal
3,0.699101,0.905486,0.887570,0.863964,0.831140,0.741161,0.822609,0.855924,basal
4,0.784501,0.797183,0.785298,0.706004,0.751453,0.709091,0.652260,0.776237,basal
5,0.631427,0.603391,0.210656,0.667289,0.760987,0.611453,0.800364,0.874278,basal
6,0.636124,0.633251,0.633291,0.824992,0.471412,0.656401,0.711842,0.679296,basal
7,0.461942,0.548796,0.780903,0.974432,0.687408,0.868626,0.684731,0.774116,basal
8,0.586140,0.698069,0.796531,0.798809,0.578886,0.660725,0.753787,0.783424,basal
9,0.698112,0.605476,0.519435,0.861607,0.887042,0.655398,0.861908,0.780554,basal


In [33]:
#perform two-way ANOVA
model = ols('height ~ C(water) + C(sun) + C(water):C(sun)', data=df ).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(water),8.533333,1.0,16.0000,0.000527
C(sun),24.866667,2.0,23.3125,0.000002
C(water):C(sun),2.466667,2.0,2.3125,0.120667
Residual,12.800000,24.0,NaN,NaN


In [11]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
import matplotlib.pyplot as plt
from scipy import stats
#### so for each dataframe.....
# degrees of freedom
data = TestParams_dict['Area Under Normalized Cross-Correlation']

N = len(data['GPR37L_1']) # variable item
degF_a = len(data['Chemical_Type'].unique()) - 1 ### factor a, string , Chemical Type
degF_b = len(data['unt1'].unique()) -1 ## factor b, numeric, Control
degF_axb = degF_a *degF_b
degF_w = N - (len(data['Chemical_Type'].unique())*len(data['unt1'].unique()))
degF_w

-24

In [12]:
## sum of squares
## mean on dependent variable 
grand_mean_DV = data['GPR37L_1'].mean()

ssq_a = sum([(data[data['Chemical_Type']==l]['GPR37L_1'].mean() - grand_mean_DV)**2 for l in data['Chemical_Type']])
ssq_b = sum([(data[data['unt1'] ==l]['GPR37L_1'].mean()-grand_mean_DV)**2 for l in data['unt1']])


ssq_t = sum((data['GPR37L_1'] - grand_mean_DV)**2)


ssq_a, ssq_b

(0.010138219415999992, 0.3501503394545)

In [27]:
# Create empty list to store the squared residuals
squared_residuals = []

# Get unique levels of unt1
unique_unt1 = data['unt1'].unique()

# Loop through each combination of Chemical_Type and unt1
for chemical_type in ['basal', 'bicucilin']:
    for unt1 in unique_unt1:
        # Get the subset of data for this combination
        subset = data[(data['Chemical_Type'] == chemical_type) & (data['unt1'] == unt1)]
        # Calculate the group mean for this combination
        group_mean = subset['GPR37L_1'].mean()
        # Calculate the squared residuals for this group and extend the list
        squared_residuals.extend((subset['GPR37L_1'] - group_mean)**2)
# Sum the squared residuals to get ssq_w
ssq_w = sum(squared_residuals)

# Print the result
print('SSQ Within:', ssq_w)

0.809806
0.890126
0.682422
0.699101
0.784501
0.631427
0.636124
0.461942
0.58614
0.698112
0.825891
0.667393
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.877927
0.898627
0.657459
0.707698
0.890765
0.677754
0.62046
0.479595
0.637769
0.756384
0.854394
0.807425
SSQ Within: 0.0


In [14]:
## sum of squares interaction
ssq_axb = ssq_t-ssq_a-ssq_b-ssq_w
ssq_axb

-0.010138219416000016

In [19]:
# Mean Square A
ms_a = ssq_a/degF_a

# Mean Square B
ms_b = ssq_b/degF_b

# Mean Square AxB
ms_axb = ssq_axb/degF_axb

# Mean Square Within/Error/Residual
ms_w = ssq_w/degF_w
ms_a

0.010138219415999992

In [20]:
# F-ratio A
f_a = ms_a/ms_w
# F-ratio B
f_b = ms_b/ms_w
# F-ratio AxB
f_axb = ms_axb/ms_w
f_a, f_axb

/tmp/ipykernel_55921/1589440301.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  f_a = ms_a/ms_w
/tmp/ipykernel_55921/1589440301.py:4: RuntimeWarning: divide by zero encountered in scalar divide
  f_b = ms_b/ms_w
/tmp/ipykernel_55921/1589440301.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  f_axb = ms_axb/ms_w


(-inf, inf)

In [23]:
p_a = stats.f.sf(f_a, degF_a, degF_w)
p_b = stats.f.sf(f_b, degF_b, degF_w)
p_axb = stats.f.sf(f_axb, degF_axb, degF_w)
p_a


nan

In [45]:
## two way anova test
# model = ols("'GPR37L 1' ~ C(type) + unt1", data=TestParams_dict['Burst Duration - Avg (s)']).fit()
genes = ['GPR37L_1', 'GPR37L_2', 'LGI2_A12', 'SLITRK5', 'THSD7_1', 'THSD7_2']

formula = f'GPR37L_1 ~ C(Chemical_Type) + C(unt1) +C(Chemical_Type):C(unt1)'  # This formula specifies a two-way ANOVA model with interaction
model = ols(formula, data=TestParams_dict['Burst Duration - Avg (s)']).fit()
anova_table = sm.stats.anova_lm(model, typ=2)  # Type 2 ANOVA DataFrame


/home/poojaparameswaran/anaconda3/envs/dataenv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1716: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
/home/poojaparameswaran/anaconda3/envs/dataenv/lib/python3.11/site-packages/statsmodels/base/model.py:1529: RuntimeWarning: invalid value encountered in multiply
  cov_p = self.normalized_cov_params * scale
/home/poojaparameswaran/anaconda3/envs/dataenv/lib/python3.11/site-packages/statsmodels/base/model.py:1529: RuntimeWarning: invalid value encountered in multiply
  cov_p = self.normalized_cov_params * scale


ValueError: array must not contain infs or NaNs